# Introduction

This notebook presents the pre-processing of the S&P 500 data used to produce the input tensors of the neural network. <br>
For each stock, the input is a raw time series of the prices (High, Low, Open, Close). 
The output is a matrix of 4 rows and n (number of available data points) columns. <br>
The columns correspond to:
- Close(t-1)/Open(t-1)
- High(t-1)/Open(t-1)
- Low(t-1)/Open(t-1)
- Open(t)/Open(t-1)
    
<u>Remark:</u> We don't need to normalize the data since it's already of ratio of 2 prices closed to one. 

# Imports

In [ ]:
#for navigation in the folders
import os
import pathlib

from time import strptime
from datetime import datetime

from tqdm import tqdm

#for plot
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns 
import PIL
import pickle
from time import strftime

import seaborn as sns
sns.despine()


<Figure size 432x288 with 0 Axes>

# Processing 

In [ ]:
data_dir = '/stocks_data/'
directory = os.getcwd() + data_dir # path to the files
files_tags = os.listdir(directory) #these are the differents files

#this is here because hidden files are also shown in the list. 
for file in files_tags:
    if file[0] == '.':
        files_tags.remove(file)
stock_name = [file.split('_')[0] for file in files_tags]
stocks = [file for file in files_tags]
print(len(stock_name) == len(stocks))
print('There are {} different stocks.'.format(len(stock_name)))

True
There are 5 different stocks.


In [ ]:
kept_stocks = list()
not_kept_stocks = list()

for s in tqdm(stocks):
    df = pd.read_csv(os.getcwd() + data_dir + s)
    kept_stocks.append(s)

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 185.20it/s]


In [ ]:
list_open = list()
list_close = list()
list_high = list()
list_low = list()

for s in tqdm(kept_stocks):
    data = pd.read_csv(os.getcwd() + data_dir + s).fillna('bfill').copy()
    data = data[['open', 'close', 'high', 'low']]
    list_open.append(data.open.values)
    list_close.append(data.close.values)
    list_high.append(data.high.values)
    list_low.append(data.low.values)

array_open = np.transpose(np.array(list_open))[:-1]
array_open_of_the_day = np.transpose(np.array(list_open))[1:]
array_close = np.transpose(np.array(list_close))[:-1]
array_high = np.transpose(np.array(list_high))[:-1]
array_low = np.transpose(np.array(list_low))[:-1]

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 161.28it/s]


In [ ]:
X = np.transpose(np.array([array_close/array_open, 
                           array_high/array_open,
                           array_low/array_open,
                           array_open_of_the_day/array_open]), axes= (0,2,1))
X.shape

(4, 5, 516)

The shape corresponds to:
- 4: Number of features
- 5: Number of stocks we want to study
- 17030: Number of data points

# Save

In [ ]:
np.save('./np_data/input.npy', X)